### 2016 - 2020 Presidential Election Comparison

In [1]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from zipfile import ZipFile

In [2]:
# Helpful constants

# Party colors
COLOR_DEM_BLUE = "#0055A4"
COLOR_REP_RED = "#E9141D"

# Cutoff for considering a county "big"
COUNTY_CARE_COUNT = 100000

# According to https://ballotpedia.org/Presidential_battleground_states,_2020
# These states were deemed to be competitive in the 2020 election:
BATTLEGROUND_STATES = ["arizona", "florida", "georgia", "iowa", "michigan",
                       "minnesota", "nevada", "new hampshire", "north carolina", 
                       "ohio", "pennsylvania", "texas", "wisconsin"]

# These states went to Trump in 2016, and Biden in 2020:
FLIPPED_STATES = ["pennsylvania", "wisconsin", "michigan", "georgia", "arizona"]

# Column names -- mainly for after cleaning when the names are normalized
COL_ST = "state"
COL_CND = "candidate"
COL_VOTES = "votes"
COL_CNTY = "county"
COL_VOTES_16 = "votes_2016"
COL_VOTES_20 = "votes_2020"
COL_TOT_16 = "total_2016"
COL_TOT_20 = "total_2020"
COL_PCT_16 = "pct_2016"
COL_PCT_20 = "pct_2020"
COL_VOTES_CHG = "votes_change"
COL_PCT_CHG = "percent_change"

# Candidate names, after normalzing Clinton 2016 and Biden 2020 as "the major Democratic candidate",
# Trump as "the major Republican candidate" in both elections, and everone else as "other":
CND_OTHER = "other"
CND_DEM = "the dem"
CND_REP = "trump"

In [3]:
# This is just a function I like using to get an overview of a large data frame:
def print_df_overview(df, title):        
    print(f"{title} DF Head:")
    print(df.head())
    print("----------------------------------")
    print(f"{title} DF dtypes:")
    print(df.dtypes)
    
    for col in df.columns:
        print("----------------------------------")
        print(f"{title} DF column value counts:")

        ABRIDGED_ROWS = 16
        counts = df[col].value_counts()
        if counts.size < 2*ABRIDGED_ROWS:
            # Print the counts all together
            print(col, "counts:")    
            print(counts)
        else:
            # Print top and bottom counts
            print(col, "top counts:")
            print(counts[0:ABRIDGED_ROWS])
            print(col, "bottom counts:")
            print(counts[-ABRIDGED_ROWS:])

In [4]:
# The data files are too big to naively manage in git, so they are stored as zip files.

# Unzip a file to a specified location:
def unzip(zip, where):
    with ZipFile(zip, 'r') as zObject:   
        print(f"Unzipping {zip} to {where}")
        zObject.extractall(path=where) 

# Paths for data files, before and after unzipping
path_2016 = os.path.join('.', 'resources', '2016')
path_2016_csv = os.path.join('.', 'resources', '2016', '2016-precinct-president.csv')
path_2016_zip = os.path.join('.', 'resources', '2016', '2016-precinct-president.csv.zip')
path_2020 = os.path.join('.', 'resources', '2020')
path_2020_csv = os.path.join('.', 'resources', '2020', 'PRESIDENT_precinct_general.csv')
path_2020_zip = os.path.join('.', 'resources', '2020', 'PRESIDENT_precinct_general.csv.zip')

# Unzip the data files
unzip(path_2016_zip, path_2016)
unzip(path_2020_zip, path_2020)

Unzipping .\resources\2016\2016-precinct-president.csv.zip to .\resources\2016
Unzipping .\resources\2020\PRESIDENT_precinct_general.csv.zip to .\resources\2020


In [5]:
# Read raw data -- takes several seconds

# Encoding wasn't documented for 2016 but ISO-8859-1 seems to work fine. 
# Some data types are specified here to suppress warnings
raw_2016_df = pd.read_csv(path_2016_csv, encoding="ISO-8859-1", 
                          dtype={"precinct":str, "district":str, "party":str, "candidate_fec":str, "candidate_fec_name":str, COL_VOTES:int} )

# The 2020 data actually includes the encoding and column data types (thanks!):
official_2020_dtypes = {'precinct':str,'office':str, 'party_detailed':str, 
		'party_simplified':str,'mode':str,'votes':int, 'county_name':str,
		'county_fips':str, 'jurisdiction_name':str,'jurisdiction_fips':str,
		'candidate':str, 'district':str, 'dataverse':str,'year':int,
		'stage':str, 'state':str, 'special':str, 'writein':str, 'state_po':str,
		'state_fips':str, 'state_cen':str, 'state_ic':str, 'date':str, 
		'readme_check':str,'magnitude':int}
raw_2020_df = pd.read_csv(path_2020_csv, encoding="ISO-8859-1", dtype=official_2020_dtypes )

raw_2016_df.shape, raw_2020_df.shape

((1989234, 37), (1982581, 25))

In [6]:
# Get ready to clean up
c_2016_df = raw_2016_df.copy()
c_2020_df = raw_2020_df.copy()
print("Initial:                 ", c_2016_df.shape, c_2020_df.shape)

# There are a few areas which notably DON'T require me to do any special handling:
# write-in votes, straight-ticket voting, and "statistical adjustments"
#
# Evidently write-in and straight-ticket votes have already been tabulated to the right candidate.
# It's odd that some rows don't actually specify a candidate to give the votes to, but if I try to
# do anything special for write-in and straight-ticket votes, I get further from the official totals,
# not closer. So I believe those rows are redundant with respect to candidates' vote totals.
#
# I don't know specifically what "Statistical adjustments" are or why they need their own rows,
# but simply including them consistently brings me closer to the correct totals.

# The 2016 data includes votes for a variety of "offices" with values such as "Straight Ticket",
# and "Unopposed Candidates". I'm not sure exactly what those mean, but the final count is a lot
# closer to the official value without them:
c_2016_df = c_2016_df.loc[c_2016_df["office"] == "US President"]
c_2020_df = c_2020_df.loc[c_2020_df["office"] == "US PRESIDENT"]
print("Drop other offices:      ", c_2016_df.shape, c_2020_df.shape)

# For reasons I do not fathom, there are a lot of -1 votes for various candidates,
# espescially in New Mexico. I have not been able to find out why, so I'm simply dropping them.
# Note some larger negative votes are "statistical adjustments" which are retained.
c_2016_df = c_2016_df.drop(c_2016_df[c_2016_df.votes == -1].index)
c_2020_df = c_2020_df.drop(c_2020_df[c_2020_df.votes == -1].index)
print("Drop negative votes:     ", c_2016_df.shape, c_2020_df.shape)

# Narrow down to the interesting columns.
c_2016_df = c_2016_df.loc[:, [COL_ST, "county_name", COL_CND, COL_VOTES] ]
c_2020_df = c_2020_df.loc[:, [COL_ST, "county_name", COL_CND, COL_VOTES] ]
print("Drop extraneous columns: ", c_2016_df.shape, c_2020_df.shape)

# Change a couple column names for consistency and conciseness
c_2016_df.rename(columns={"county_name": COL_CNTY}, inplace=True)
c_2020_df.rename(columns={"county_name": COL_CNTY}, inplace=True)

# 2016 uses normal capitalization while 2020 uses all caps
# So, California vs CALIFORNIA, and District of Columbia vs DISTRICT OF COLUMBIA
# To reconcile them, force everything to lowercase.
c_2016_df[COL_ST] = c_2016_df[COL_ST].str.lower()
c_2016_df[COL_CNTY] = c_2016_df[COL_CNTY].str.lower()
c_2016_df[COL_CND] = c_2016_df[COL_CND].str.lower()
c_2020_df[COL_ST] = c_2020_df[COL_ST].str.lower()
c_2020_df[COL_CNTY] = c_2020_df[COL_CNTY].str.lower()
c_2020_df[COL_CND] = c_2020_df[COL_CND].str.lower()

# County names actually say "County" in the 2016 data, so remove that
c_2016_df[COL_CNTY] = c_2016_df[COL_CNTY].str.replace(" county", "")

# Reconcile candidate names. Clinton and Biden are equivalent for our purposes, 
# and Biden and Trump both appear in multiple forms in the datasets.
# Any other candidate is just "other".
dem_names = ["hillary clinton", "biden, joe", "joseph biden", "joseph r biden"]
rep_names = ["donald trump", "donald j trump"]
combo = dem_names + rep_names
c_2016_df.loc[~c_2016_df[COL_CND].isin(combo), COL_CND] = CND_OTHER
c_2020_df.loc[~c_2020_df[COL_CND].isin(combo), COL_CND] = CND_OTHER
c_2016_df.loc[c_2016_df[COL_CND].isin(dem_names), COL_CND] = CND_DEM
c_2016_df.loc[c_2016_df[COL_CND].isin(rep_names), COL_CND] = CND_REP
c_2020_df.loc[c_2020_df[COL_CND].isin(dem_names), COL_CND] = CND_DEM
c_2020_df.loc[c_2020_df[COL_CND].isin(rep_names), COL_CND] = CND_REP

Initial:                  (1989234, 37) (1982581, 25)
Drop other offices:       (1954659, 37) (1982581, 25)
Drop negative votes:      (1954656, 37) (1975341, 25)


KeyError: "['COL_ST', 'COL_CND', 'COL_VOTES'] not in index"

In [ ]:
# Print out some info to show how much data we lost to cleaning and how it relates to offical results
def before_and_after(year, raw_df, clean_df, dem_raw, dem_official_votes, rep_raw, rep_official_votes, all_official_votes):
    raw_rows = raw_df.shape[0]
    raw_votes = raw_df['votes'].sum()
    clean_rows = clean_df.shape[0]
    clean_votes = clean_df['votes'].sum()

    dem_raw_votes = raw_df.loc[raw_df[COL_CND].str.contains(dem_raw, case=False, na=False)][COL_VOTES].sum()
    dem_clean_votes = clean_df.loc[clean_df[COL_CND] == CND_DEM][COL_VOTES].sum()
    
    rep_raw_votes = raw_df.loc[raw_df[COL_CND].str.contains(rep_raw, case=False, na=False)][COL_VOTES].sum()    
    rep_clean_votes = clean_df.loc[clean_df[COL_CND] == CND_REP][COL_VOTES].sum()
    
    print(f"{year}:")
    print(f"Original: {raw_rows:7} rows, {raw_votes:10} votes")
    print(f"Cleaned:  {clean_rows:7} rows, {clean_votes:10} votes")
    print(f"Democrat Votes (Wikipedia):   {dem_official_votes:10}")
    print(f"Democrat Votes (Raw):         {dem_raw_votes:10} ({dem_raw_votes/dem_official_votes:.3%}%)")
    print(f"Democrat Votes (Clean):       {dem_clean_votes:10} ({dem_clean_votes/dem_official_votes:.3%}%)")
    print(f"Republican Votes (Wikipedia): {rep_official_votes:10}")
    print(f"Republican Votes (Raw):       {rep_raw_votes:10} ({rep_raw_votes/rep_official_votes:.3%}%)")
    print(f"Republican Votes (Clean):     {rep_clean_votes:10} ({rep_clean_votes/rep_official_votes:.3%}%)")
    print(f"Total Votes (Wikipedia):      {all_official_votes:10}")
    print(f"Total Votes (Raw):            {raw_votes:10} ({raw_votes/all_official_votes:.3%}%)")
    print(f"Total Votes (Clean):          {clean_votes:10} ({clean_votes/all_official_votes:.3%}%)")

# Official vote numbers come from the FEC:
#
# FEDERAL ELECTIONS 2016 Election Results for the U.S. President, the U.S. Senate and the U.S. House of Representatives:
# https://www.fec.gov/resources/cms-content/documents/federalelections2016.pdf
# FEDERAL ELECTIONS 2020 Election Results for the U.S. President, the U.S. Senate and the U.S. House of Representatives:
# https://www.fec.gov/resources/cms-content/documents/federalelections2020.pdf
before_and_after(2016, raw_2016_df, c_2016_df, "hillary clinton", 65853514, CND_REP, 62984828, 136669276)
print()
before_and_after(2020, raw_2020_df, c_2020_df, "biden", 81283501, CND_REP, 74223975, 158429631)

In [ ]:
# Just some debug output for during development
#print_df_overview(c_2016_df, "2016 Cleaned")
#print_df_overview(c_2020_df, "2020 Cleaned")

In [ ]:
# The two dataframes are so large at this point that merging or joining them is problematic.
# I kept running out of memory. So we squash them down a lot first, summing up the votes for
# each candidate in each county. Combining all the CND_OTHER rows reduces dataframe size
# tremendously, making them much easier to deal with:
grouped_2016 = pd.DataFrame(c_2016_df.groupby([COL_ST, COL_CNTY, COL_CND])[COL_VOTES].sum())
grouped_2020 = pd.DataFrame(c_2020_df.groupby([COL_ST, COL_CNTY, COL_CND])[COL_VOTES].sum())
combo_df = pd.merge(grouped_2016, grouped_2020, on=[COL_ST, COL_CNTY, COL_CND], suffixes=["_2016", "_2020"])

# The actions above result in a "multi index" dataframe, which I don't really want.
# So do regular old simple indexing:
combo_df.reset_index(inplace=True)

print_df_overview(combo_df, "Combo DF")

In [ ]:
# Add vote totals columns
combo_df[COL_TOT_16] = ""
combo_df[COL_TOT_20] = ""

for state in combo_df[COL_ST].unique():
    state_slice = combo_df.loc[combo_df[COL_ST] == state]
    counties = state_slice[COL_CNTY].unique()
    for county in counties:
        county_slice = state_slice.loc[combo_df[COL_CNTY] == county]        
        total_2016 = county_slice[COL_VOTES_16].sum()
        total_2020 = county_slice[COL_VOTES_20].sum()

        combo_df.loc[county_slice.index, COL_TOT_16] = total_2016
        combo_df.loc[county_slice.index, COL_TOT_20] = total_2020

combo_df

In [ ]:
# Add the columns showing percentages and changes from 2016-2020
combo_df[COL_PCT_16] = combo_df[COL_VOTES_16] / combo_df[COL_TOT_16]
combo_df[COL_PCT_20] = combo_df[COL_VOTES_20] / combo_df[COL_TOT_20]
combo_df[COL_VOTES_CHG] = combo_df[COL_VOTES_20] - combo_df[COL_VOTES_16]
combo_df[COL_PCT_CHG] = combo_df[COL_PCT_20] - combo_df[COL_PCT_16]

# Save to CSV to enable some manual sanity checks
combo_df.to_csv("combined.csv")

# Make the DF for the biggest counties
big_counties_df = combo_df.loc[combo_df[COL_TOT_16] >= COUNTY_CARE_COUNT].loc[combo_df[COL_TOT_20] >= COUNTY_CARE_COUNT]

# Note many counties we have data for. This doesn't impact any calculation directly, but is nice to know:
county_count = len(combo_df[COL_CNTY].unique())
big_county_count = len(big_counties_df[COL_CNTY].unique())
county_count, big_county_count

In [ ]:
# I have a lot of boxplots, and this little function is for adding median values as text
# just above the median line in the box:
def add_percent_above_value(subplot, x, ylim, value, digits=0, color="black"):
    range = ylim[1] - ylim[0]
    voffset = range * 0.015
    subplot.text(x, value + voffset, f"{value:.{digits}%}", va='center', ha='center', color=color, weight='bold')  

In [ ]:
# This function makes a boxplot showing national, all county, and big county percentages
# for both election years:
def plot_result_comparisons(subplot, candidate_str, title, color):
    national_gb = combo_df.groupby(COL_CND).sum()
    candidate = candidate_str

    national_pct_2016 = national_gb[COL_VOTES_16][candidate_str] / national_gb[COL_VOTES_16].sum()
    national_pct_2020 = national_gb[COL_VOTES_20][candidate_str] / national_gb[COL_VOTES_20].sum()

    combo_slice = combo_df.loc[combo_df[COL_CND] == candidate_str]
    big_slice = big_counties_df.loc[big_counties_df[COL_CND] == candidate_str]

    subplot.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:.0%}"))

    subplot.boxplot(
            [national_pct_2016, combo_slice[COL_PCT_16], big_slice[COL_PCT_16],
             national_pct_2020, combo_slice[COL_PCT_20], big_slice[COL_PCT_20]],
            tick_labels = ["2016\nNational", f"2016 All\nCounties\n({county_count})", f"2016 Big\nCounties\n({big_county_count})", 
                           "2020\nNational", f"2020 All\nCounties\n({county_count})", f"2020 Big\nCounties\n({big_county_count})"],
            flierprops= {"alpha": 0.33} )
    subplot.set_title(f"People Voting for {title}")
    subplot.set_ylabel(f"Voted for {title} (%)")
    subplot.set_ylim((0,1))
    subplot.grid(axis="y", which="major", color="lightgrey")

    # Divide the years
    subplot.axvline(3.5, color=color, alpha=0.33)
    subplot.text(2, 0.98, "2016", va="center", ha="center", color=color, weight="bold")
    subplot.text(5, 0.98, "2020", va="center", ha="center", color=color, weight="bold")

    add_percent_above_value(subplot, 1, subplot.get_ylim(), national_pct_2016, color=color)
    add_percent_above_value(subplot, 2, subplot.get_ylim(), combo_slice[COL_PCT_16].median(), color=color)
    add_percent_above_value(subplot, 3, subplot.get_ylim(), big_slice[COL_PCT_16].median(), color=color)
    add_percent_above_value(subplot, 4, subplot.get_ylim(), national_pct_2020, color=color)
    add_percent_above_value(subplot, 5, subplot.get_ylim(), combo_slice[COL_PCT_20].median(), color=color)
    add_percent_above_value(subplot, 6, subplot.get_ylim(), big_slice[COL_PCT_20].median(), color=color)

# Side-by-size boxplots for results for both major candidates:
fig, (ax1, ax2) = plt.subplots(nrows=1,ncols=2,figsize=(12,6))
plot_result_comparisons(ax1, CND_DEM, "the Democrat", COLOR_DEM_BLUE)
plot_result_comparisons(ax2, CND_REP, "the Republican", COLOR_REP_RED)
plt.show()

In [ ]:
# Show a boxplot for people voting for CND_OTHER candidates, meaning anyone but the two
# main party nominees, in 2016 and 2020.
combo_slice = combo_df.loc[combo_df[COL_CND] == CND_OTHER]
big_slice = big_counties_df.loc[big_counties_df[COL_CND] == CND_OTHER]

plt.axes().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:.0%}"))
plt.boxplot(
        [combo_slice[COL_PCT_16], big_slice[COL_PCT_16],
         combo_slice[COL_PCT_20], big_slice[COL_PCT_20]],
        tick_labels = ["2016 All\nCounties", "2016 Big\nCounties", 
                       "2020 All\nCounties", "2020 Big\nCounties"],
        flierprops= {"alpha": 0.33} )
plt.title("Fewer People Voted for Minor Candidates in 2020")
plt.ylabel("Voted For Other (%)")
plt.ylim((0,0.15))
plt.grid(axis="y", which="major", color="lightgrey")
plt.axvline(2.5, color="lightgrey")

add_percent_above_value(plt, 1, plt.ylim(), combo_slice[COL_PCT_16].median(), digits=1)
add_percent_above_value(plt, 2, plt.ylim(), big_slice[COL_PCT_16].median(), digits=1)
add_percent_above_value(plt, 3, plt.ylim(), combo_slice[COL_PCT_20].median(), digits=1)
add_percent_above_value(plt, 4, plt.ylim(), big_slice[COL_PCT_20].median(), digits=1)

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1,ncols=2,figsize=(12,6))

ax1.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:+.0%}"))
ax1.boxplot(
        [combo_df.loc[combo_df[COL_CND] == CND_DEM][COL_PCT_CHG], 
         combo_df.loc[combo_df[COL_CND] == CND_REP][COL_PCT_CHG],
         combo_df.loc[combo_df[COL_CND] == CND_OTHER][COL_PCT_CHG]],
        tick_labels = ["Democrat", "Republican", "Non-Major"],
        showfliers=False )
ax1.set_title("Across All Counties, Both Candidates Gained Similarly")
ax1.set_ylabel("Change in Votes 2016 - 2020")
ax1.set_xlabel("---All Counties---")
ax1.set_ylim((-.1,.1))
ax1.grid(axis="y", which="major", color="lightgrey")


add_percent_above_value(ax1, 1, ax1.get_ylim(), combo_df.loc[combo_df[COL_CND] == CND_DEM][COL_PCT_CHG].median(), digits=1, color=COLOR_DEM_BLUE)
add_percent_above_value(ax1, 2, ax1.get_ylim(), combo_df.loc[combo_df[COL_CND] == CND_REP][COL_PCT_CHG].median(), digits=1, color=COLOR_REP_RED)
add_percent_above_value(ax1, 3, ax1.get_ylim(), combo_df.loc[combo_df[COL_CND] == CND_OTHER][COL_PCT_CHG].median(), digits=1)

ax2.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:.0%}"))
ax2.boxplot(
        [big_counties_df.loc[big_counties_df[COL_CND] == CND_DEM][COL_PCT_CHG], 
         big_counties_df.loc[big_counties_df[COL_CND] == CND_REP][COL_PCT_CHG],
         big_counties_df.loc[big_counties_df[COL_CND] == CND_OTHER][COL_PCT_CHG]],
        tick_labels = ["Democrat", "Republican", "Non-Major"],
        showfliers=False )
ax2.set_title("Big Counties Helped the Democrats")
ax2.set_ylabel("Change in Votes 2016 - 2020")
ax2.set_xlabel("----Big Counties----")
ax2.set_ylim((-.1,.1))
ax2.grid(axis="y", which="major", color="lightgrey")

add_percent_above_value(ax2, 1, ax2.get_ylim(), big_counties_df.loc[big_counties_df[COL_CND] == CND_DEM][COL_PCT_CHG].median(), digits=1, color=COLOR_DEM_BLUE)
add_percent_above_value(ax2, 2, ax2.get_ylim(), big_counties_df.loc[big_counties_df[COL_CND] == CND_REP][COL_PCT_CHG].median(), digits=1, color=COLOR_REP_RED)
add_percent_above_value(ax2, 3, ax2.get_ylim(), big_counties_df.loc[big_counties_df[COL_CND] == CND_OTHER][COL_PCT_CHG].median(), digits=1)
plt.show()

In [ ]:
#Scatterplot showing relationship between county size and gain for party
def scatter_party(subplot, candidate_str, title, color, trendcolor, log):
    slice = combo_df.loc[combo_df[COL_CND] == candidate_str]
    x = slice[COL_TOT_20] / 1000
    y = slice[COL_PCT_CHG]
    subplot.set_title(title)

    subplot.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:+.1%}"))    
    subplot.set_title(title)
    subplot.set_ylabel("Change in Votes")
    subplot.set_xlabel("County Size (2020 Vote Total)")
    subplot.set_ylim((-.5, 0.5))

    if log:
        subplot.scatter(x, y, color=color, alpha=0.05)
        subplot.set_xscale("log")
    else:
        subplot.scatter(x, y, color=color, alpha=0.33)
        subplot.set_xscale("linear") # AKA normal
    
        z = np.polyfit(list(x), list(y), 1)
        p = np.poly1d(z)
        subplot.plot(x,p(x),trendcolor)

    subplot.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:.0f} K"))

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2,ncols=2,figsize=(15,12))
scatter_party(ax1, CND_DEM, "Democratic Candidate", COLOR_DEM_BLUE, "black", False)
scatter_party(ax2, CND_REP, "Republican Candidate", COLOR_REP_RED, "black", False)
scatter_party(ax3, CND_DEM, "Democratic Candidate -- Log Scale", COLOR_DEM_BLUE, "black", True)
scatter_party(ax4, CND_REP, "Republican Candidate -- Log Scale", COLOR_REP_RED, "black", True)
plt.show()

In [ ]:
# Show shift betwen dem / rep / other votes in battleground states, flipped states, and the whole country

battleground_slice = combo_df.loc[combo_df[COL_ST].isin(BATTLEGROUND_STATES)]
flipped_slice = combo_df.loc[combo_df[COL_ST].isin(FLIPPED_STATES)]

fig, (ax1, ax2, ax3) = plt.subplots(nrows=1,ncols=3,figsize=(18,6))

ax1.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:+.0%}"))
ax1.boxplot(
        [battleground_slice.loc[battleground_slice[COL_CND] == CND_DEM][COL_PCT_CHG], 
         battleground_slice.loc[battleground_slice[COL_CND] == CND_REP][COL_PCT_CHG],
         battleground_slice.loc[battleground_slice[COL_CND] == CND_OTHER][COL_PCT_CHG]],
        tick_labels = ["Democrat", "Republican", "Non-Major"],
        showfliers=False )
ax1.set_title("Counties in Battleground States")
ax1.set_ylabel("Change in Votes 2016 - 2020")
ax1.set_xlabel("---Counties in Battleground States---")
ax1.set_ylim((-.1,.1))
ax1.grid(axis="y", which="major", color="lightgrey")


add_percent_above_value(ax1, 1, ax1.get_ylim(), battleground_slice.loc[battleground_slice[COL_CND] == CND_DEM][COL_PCT_CHG].median(), digits=1, color=COLOR_DEM_BLUE)
add_percent_above_value(ax1, 2, ax1.get_ylim(), battleground_slice.loc[battleground_slice[COL_CND] == CND_REP][COL_PCT_CHG].median(), digits=1, color=COLOR_REP_RED)
add_percent_above_value(ax1, 3, ax1.get_ylim(), battleground_slice.loc[battleground_slice[COL_CND] == CND_OTHER][COL_PCT_CHG].median(), digits=1)


ax2.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:.0%}"))
ax2.boxplot(
        [flipped_slice.loc[flipped_slice[COL_CND] == CND_DEM][COL_PCT_CHG], 
         flipped_slice.loc[flipped_slice[COL_CND] == CND_REP][COL_PCT_CHG],
         flipped_slice.loc[flipped_slice[COL_CND] == CND_OTHER][COL_PCT_CHG]],
        tick_labels = ["Democrat", "Republican", "Non-Major"],
        showfliers=False )
ax2.set_title("Counties in Flipped States")
ax2.set_ylabel("Change in Votes 2016 - 2020")
ax2.set_xlabel("----Counties in Flipped States----")
ax2.set_ylim((-.1,.1))
ax2.grid(axis="y", which="major", color="lightgrey")

add_percent_above_value(ax2, 1, ax2.get_ylim(), flipped_slice.loc[flipped_slice[COL_CND] == CND_DEM][COL_PCT_CHG].median(), digits=1, color=COLOR_DEM_BLUE)
add_percent_above_value(ax2, 2, ax2.get_ylim(),  flipped_slice.loc[flipped_slice[COL_CND] == CND_REP][COL_PCT_CHG].median(), digits=1, color=COLOR_REP_RED)
add_percent_above_value(ax2, 3, ax2.get_ylim(), flipped_slice.loc[flipped_slice[COL_CND] == CND_OTHER][COL_PCT_CHG].median(), digits=1)


ax3.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:.0%}"))
ax3.boxplot(
        [combo_df.loc[combo_df[COL_CND] == CND_DEM][COL_PCT_CHG], 
         combo_df.loc[combo_df[COL_CND] == CND_REP][COL_PCT_CHG],
         combo_df.loc[combo_df[COL_CND] == CND_OTHER][COL_PCT_CHG]],
        tick_labels = ["Democrat", "Republican", "Non-Major"],
        showfliers=False )
ax3.set_title("Counties in All States")
ax3.set_ylabel("Change in Votes 2016 - 2020")
ax3.set_xlabel("----Counties in All States----")
ax3.set_ylim((-.1,.1))
ax3.grid(axis="y", which="major", color="lightgrey")

add_percent_above_value(ax3, 1, ax3.get_ylim(), combo_df.loc[combo_df[COL_CND] == CND_DEM][COL_PCT_CHG].median(), digits=1, color=COLOR_DEM_BLUE)
add_percent_above_value(ax3, 2, ax3.get_ylim(), combo_df.loc[combo_df[COL_CND] == CND_REP][COL_PCT_CHG].median(), digits=1, color=COLOR_REP_RED)
add_percent_above_value(ax3, 3, ax3.get_ylim(), combo_df.loc[combo_df[COL_CND] == CND_OTHER][COL_PCT_CHG].median(), digits=1)
plt.show()